In [171]:
import pandas as pd
import numpy as np

In [172]:
prefix  = "../data/"

df_demographic = pd.read_csv(prefix+"DemographicData_ZCTAs.csv")
df_economic = pd.read_csv(prefix+"EconomicData_ZCTAs.csv")
df_geocode = pd.read_csv(prefix+"df_geocode.csv")
df_tests = pd.read_csv(prefix+"test_data.csv")
df_transactional = pd.read_csv(prefix+"transactional_data.csv")

# Sanity check

## Sanity check df_demographic

In [173]:
df_demographic.head(5)

,Unnamed: 0,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
0,0,8600000US35004,ZCTA5 35004,12045,94.1,805,1075,898,477,578,2088,1628,1200,886,683,1017,534,176,35.5
1,1,8600000US35005,ZCTA5 35005,7344,86.1,504,453,511,499,214,788,795,968,612,561,798,485,156,44.0
2,2,8600000US35006,ZCTA5 35006,2883,108.2,96,153,303,129,156,183,367,430,296,260,280,201,29,47.2
3,3,8600000US35007,ZCTA5 35007,26332,95.0,1936,1992,1837,1762,1376,3119,3849,3907,1665,1323,2096,1106,364,37.7
4,4,8600000US35010,ZCTA5 35010,20613,90.5,1306,1465,944,1217,1128,2513,2106,2950,1512,1472,2421,1155,424,42.6


Corrigindo o nome das colunas

In [174]:
df_demographic.columns = [x.replace(" ","").replace("(","_").replace(")","") for x in df_demographic.columns] # Renomeando as colunas 

Descartando a coluna Unnamed:0, pois já o dataframe já contém index

In [175]:
df_demographic = df_demographic.iloc[:,1:]

Verificando as colunas com valores nulos

In [176]:
df_demographic.isnull().any()

Id                             False
GeographicAreaName             False
TotalPopulation                False
SexRatio_malesper100females     True
Population_Under5Years         False
Population_5to9Years           False
Population_10to14Years         False
Population_15to19Years         False
Population_20to24Years         False
Population_25to34Years         False
Population_35to44Years         False
Population_45to54Years         False
Population_55to59Years         False
Population_60to64Years         False
Population_65to74Years         False
Population_75to84Years         False
Population_85YearsAndOver      False
MedianAgeInYears                True
dtype: bool

In [177]:
print(f"Como vemos, cerca de {((100*df_demographic[df_demographic.isna().any(axis=1)].shape[0])/df_demographic.shape[0]):.3f}\
% das {df_demographic.shape[0]} linhas contém dados nulos")

Como vemos, cerca de 1.863% das 33120 linhas contém dados nulos


Como não é possível obter os dados faltantes, as linhas que contém os dados faltantes serão excluidas do dataframe

In [178]:
df_demographic = df_demographic.dropna(how='any') # Dropando os valores nulos

Como é possível verificar, não constam mais valores nulos em nenhuma coluna do dataframe

In [179]:
df_demographic.isnull().any()

Id                             False
GeographicAreaName             False
TotalPopulation                False
SexRatio_malesper100females    False
Population_Under5Years         False
Population_5to9Years           False
Population_10to14Years         False
Population_15to19Years         False
Population_20to24Years         False
Population_25to34Years         False
Population_35to44Years         False
Population_45to54Years         False
Population_55to59Years         False
Population_60to64Years         False
Population_65to74Years         False
Population_75to84Years         False
Population_85YearsAndOver      False
MedianAgeInYears               False
dtype: bool

Corrigindo o index após a remoção dos valores nulos

In [180]:
df_demographic = df_demographic.reset_index(drop=True)

Verificando os dtypes das colunas 

In [181]:
df_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32503 entries, 0 to 32502
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           32503 non-null  object 
 1   GeographicAreaName           32503 non-null  object 
 2   TotalPopulation              32503 non-null  int64  
 3   SexRatio_malesper100females  32503 non-null  float64
 4   Population_Under5Years       32503 non-null  int64  
 5   Population_5to9Years         32503 non-null  int64  
 6   Population_10to14Years       32503 non-null  int64  
 7   Population_15to19Years       32503 non-null  int64  
 8   Population_20to24Years       32503 non-null  int64  
 9   Population_25to34Years       32503 non-null  int64  
 10  Population_35to44Years       32503 non-null  int64  
 11  Population_45to54Years       32503 non-null  int64  
 12  Population_55to59Years       32503 non-null  int64  
 13  Population_60to6

É possível verificar que apenas as colunas "Id" e "GeographicAreaName" são do tipo "object", e serão convertidas para tipos específicos

In [182]:
df_demographic["Id"] = df_demographic["Id"].astype(pd.StringDtype())
df_demographic["GeographicAreaName"] = df_demographic["GeographicAreaName"].astype(pd.StringDtype())

Com isso, todos os Dtype(s) do dataframe foram corrigidos

In [184]:
df_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32503 entries, 0 to 32502
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           32503 non-null  string 
 1   GeographicAreaName           32503 non-null  string 
 2   TotalPopulation              32503 non-null  int64  
 3   SexRatio_malesper100females  32503 non-null  float64
 4   Population_Under5Years       32503 non-null  int64  
 5   Population_5to9Years         32503 non-null  int64  
 6   Population_10to14Years       32503 non-null  int64  
 7   Population_15to19Years       32503 non-null  int64  
 8   Population_20to24Years       32503 non-null  int64  
 9   Population_25to34Years       32503 non-null  int64  
 10  Population_35to44Years       32503 non-null  int64  
 11  Population_45to54Years       32503 non-null  int64  
 12  Population_55to59Years       32503 non-null  int64  
 13  Population_60to6

Tratamento de outliers presentes no dataframe

## Sanity check df_economic

In [192]:
df_economic.head(5)

,Unnamed: 0,id,Geographic Area Name,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore
0,0,8600000US35004,ZCTA5 35004,198,71,298,513,647,1117,529,945,245,61
1,1,8600000US35005,ZCTA5 35005,188,184,318,293,353,562,299,407,67,26
2,2,8600000US35006,ZCTA5 35006,71,20,117,104,154,176,124,194,51,7
3,3,8600000US35007,ZCTA5 35007,396,208,670,462,1173,1854,1578,2224,473,254
4,4,8600000US35010,ZCTA5 35010,700,610,1093,957,1056,1512,807,749,254,249


Corrigindo o nome das colunas

In [193]:
df_economic.columns = [x.replace(" ","").replace("(","_").replace(")","") for x in df_economic.columns] # Renomeando as colunas 

ValueError: Length mismatch: Expected axis has 13 elements, new values have 18 elements